In [1]:
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt
from matplotlib import patches as patches
import cv2

In [42]:
train_data = pd.read_json("C:/Users/Adam/Desktop/ISA/Tensorflow_2_object_detection_API/malaria/training.json")
test_data = pd.read_json("C:/Users/Adam/Desktop/ISA/Tensorflow_2_object_detection_API/malaria/test.json")
print(train_data.shape, test_data.shape)

(1208, 2) (120, 2)


In [43]:
train_data.head()

,image,objects
0,{'checksum': '676bb8e86fc2dbf05dd97d51a64ac0af...,"[{'bounding_box': {'minimum': {'r': 1057, 'c':..."
1,{'checksum': '1225a18efce159eddf7b0e80e0ea642c...,"[{'bounding_box': {'minimum': {'r': 734, 'c': ..."
2,{'checksum': '3eaf840523c30fdf38897ffa01e194eb...,"[{'bounding_box': {'minimum': {'r': 724, 'c': ..."
3,{'checksum': '8a111dffacfa433029492780b9535091...,"[{'bounding_box': {'minimum': {'r': 563, 'c': ..."
4,{'checksum': 'ccef403e971460b86444cca669e68ca1...,"[{'bounding_box': {'minimum': {'r': 618, 'c': ..."


In [44]:
print(train_data['image'].sample().values, "\n",#
      train_data['objects'].sample().values)

[{'checksum': '7cf64fe4bccbe42257a67370d01f5ae8', 'pathname': '/images/bcaefb02-682a-41b9-acc7-e6cc010f5b30.png', 'shape': {'r': 1200, 'c': 1600, 'channels': 3}}] 
 [list([{'bounding_box': {'minimum': {'r': 637, 'c': 746}, 'maximum': {'r': 763, 'c': 867}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 641, 'c': 127}, 'maximum': {'r': 741, 'c': 246}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 369, 'c': 855}, 'maximum': {'r': 488, 'c': 960}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 802, 'c': 551}, 'maximum': {'r': 906, 'c': 653}}, 'category': 'leukocyte'}, {'bounding_box': {'minimum': {'r': 853, 'c': 1436}, 'maximum': {'r': 970, 'c': 1560}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 501, 'c': 809}, 'maximum': {'r': 618, 'c': 932}}, 'category': 'red blood cell'}, {'bounding_box': {'minimum': {'r': 734, 'c': 1426}, 'maximum': {'r': 854, 'c': 1550}}, 'category': 'red blood cell'}, {'bounding_box': 

In [45]:
#splitting train set into train and validation set
#https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas

msk = np.random.rand(len(train_data)) < 0.8

train = train_data[msk]
valid = train_data[~msk]

In [46]:
train.reset_index(inplace=True)
valid.reset_index(inplace=True)

In [47]:
train.shape, valid.shape

((952, 3), (256, 3))

In [48]:
train.head()

,index,image,objects
0,1,{'checksum': '1225a18efce159eddf7b0e80e0ea642c...,"[{'bounding_box': {'minimum': {'r': 734, 'c': ..."
1,2,{'checksum': '3eaf840523c30fdf38897ffa01e194eb...,"[{'bounding_box': {'minimum': {'r': 724, 'c': ..."
2,3,{'checksum': '8a111dffacfa433029492780b9535091...,"[{'bounding_box': {'minimum': {'r': 563, 'c': ..."
3,6,{'checksum': '36f63469b09e117ade01d97d3c7e2120...,"[{'bounding_box': {'minimum': {'r': 124, 'c': ..."
4,7,{'checksum': '82d98ef5d434e1b2d4693baef3d6c30e...,"[{'bounding_box': {'minimum': {'r': 130, 'c': ..."


#### Creating annotation csv file for training purpose
It should be in the below form:
filename, width, height, class, x_min, y_min, x_max, y_max

In [49]:
# creating dataframe with each bounding box as datapoint
train_image_list = [] # pathname
width = []            # width of the image
height = []           # height of the image
    # position of the bbox
y_max = []
y_min = []
x_max = []
x_min = []
    # label
class_label = []

for i in range(0,train.shape[0]):
    for j in range(0,len(train["objects"][i])):
        
        train_image_list.append(train["image"][i]["pathname"])
        
        width.append(train["image"][i]["shape"]["c"])
        height.append(train["image"][i]["shape"]["r"])
        
        y_max.append(train["objects"][i][j]["bounding_box"]["maximum"]["r"])
        y_min.append(train["objects"][i][j]["bounding_box"]["minimum"]["r"])
        x_max.append(train["objects"][i][j]["bounding_box"]["maximum"]["c"])
        x_min.append(train["objects"][i][j]["bounding_box"]["minimum"]["c"])
        
        class_label.append(train["objects"][i][j]["category"])
        
train_df = pd.DataFrame()

train_df["filepath"] = train_image_list
train_df["width"] = width
train_df["height"] = height
train_df["class"] = class_label
train_df["xmin"] = x_min
train_df["ymin"] = y_min
train_df["xmax"] = x_max
train_df["ymax"] = y_max

train_df.head()

,filepath,width,height,class,xmin,ymin,xmax,ymax
0,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,735,734,834,832
1,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1283,939,1378,1039
2,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1134,367,1235,476
3,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,766,307,864,400
4,/images/10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1463,596,1556,701


In [50]:
train_df["filepath"].nunique()

952

In [51]:
train_df["class"].value_counts()

red blood cell    61154
trophozoite        1174
difficult           334
ring                278
schizont            139
gametocyte          114
leukocyte            77
Name: class, dtype: int64

In [52]:
# create func to change filepath to filename
def create_filename(filepath):
    filepath = str(filepath)
    _, _, filename = filepath.split("/")
    
    return filename  

train_df.filepath = train_df.filepath.apply(create_filename)
train_df.rename(columns={"filepath": "filename"}, inplace=True)
train_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,735,734,834,832
1,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1283,939,1378,1039
2,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1134,367,1235,476
3,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,766,307,864,400
4,10be6380-cbbb-4886-8b9e-ff56b1710576.png,1600,1200,red blood cell,1463,596,1556,701


In [54]:
train_df.to_csv("images/annotation_train.csv",index = False)

#### Creating annotation csv file for validation purpose
It should be in the below form:
filename, width, height, class, x_min, y_min, x_max, y_max

In [55]:
# creating dataframe with each bounding box as datapoint
valid_image_list = [] # pathname
width = []            # width of the image
height = []           # height of the image
    # position of the bbox
y_max = []
y_min = []
x_max = []
x_min = []
    # label
class_label = []

for i in range(0,valid.shape[0]):
    for j in range(0,len(valid["objects"][i])):
        
        valid_image_list.append(valid["image"][i]["pathname"])
        
        width.append(valid["image"][i]["shape"]["c"])
        height.append(valid["image"][i]["shape"]["r"])
        
        y_max.append(valid["objects"][i][j]["bounding_box"]["maximum"]["r"])
        y_min.append(valid["objects"][i][j]["bounding_box"]["minimum"]["r"])
        x_max.append(valid["objects"][i][j]["bounding_box"]["maximum"]["c"])
        x_min.append(valid["objects"][i][j]["bounding_box"]["minimum"]["c"])
        
        class_label.append(valid["objects"][i][j]["category"])
        
valid_df = pd.DataFrame()

valid_df["filepath"] = valid_image_list
valid_df["width"] = width
valid_df["height"] = height
valid_df["class"] = class_label
valid_df["xmin"] = x_min
valid_df["ymin"] = y_min
valid_df["xmax"] = x_max
valid_df["ymax"] = y_max

valid_df.head()

,filepath,width,height,class,xmin,ymin,xmax,ymax
0,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,1440,1057,1540,1158
1,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,1303,868,1403,971
2,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,900,578,1008,689
3,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,611,304,713,408
4,/images/8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,881,198,1003,312


In [56]:
valid_df["filepath"].nunique()

256

In [57]:
valid_df["class"].value_counts()

red blood cell    16266
trophozoite         299
difficult           107
ring                 75
schizont             40
gametocyte           30
leukocyte            26
Name: class, dtype: int64

In [58]:
valid_df.filepath = valid_df.filepath.apply(create_filename)
valid_df.rename(columns={"filepath": "filename"}, inplace=True)
valid_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,1440,1057,1540,1158
1,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,1303,868,1403,971
2,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,900,578,1008,689
3,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,611,304,713,408
4,8d02117d-6c71-4e47-b50a-6cc8d5eb1d55.png,1600,1200,red blood cell,881,198,1003,312


In [59]:
valid_df.to_csv("images/annotation_valid.csv",index = False)

#### Creating annotation csv file for testing purpose
It should be in the below form:
filename, width, height, class, x_min, y_min, x_max, y_max

In [61]:
# creating dataframe with each bounding box as datapoint
test_image_list = [] # pathname
width = []            # width of the image
height = []           # height of the image
    # position of the bbox
y_max = []
y_min = []
x_max = []
x_min = []
    # label
class_label = []

for i in range(0,test_data.shape[0]):
    for j in range(0,len(test_data["objects"][i])):
        
        test_image_list.append(test_data["image"][i]["pathname"])
        
        width.append(test_data["image"][i]["shape"]["c"])
        height.append(test_data["image"][i]["shape"]["r"])
        
        y_max.append(test_data["objects"][i][j]["bounding_box"]["maximum"]["r"])
        y_min.append(test_data["objects"][i][j]["bounding_box"]["minimum"]["r"])
        x_max.append(test_data["objects"][i][j]["bounding_box"]["maximum"]["c"])
        x_min.append(test_data["objects"][i][j]["bounding_box"]["minimum"]["c"])
        
        class_label.append(test_data["objects"][i][j]["category"])
        
test_df = pd.DataFrame()

test_df["filepath"] = test_image_list
test_df["width"] = width
test_df["height"] = height
test_df["class"] = class_label
test_df["xmin"] = x_min
test_df["ymin"] = y_min
test_df["xmax"] = x_max
test_df["ymax"] = y_max

test_df.head()

,filepath,width,height,class,xmin,ymin,xmax,ymax
0,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1744,576,1883,708
1,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1249,863,1373,977
2,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1573,210,1711,335
3,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1124,1017,1253,1142
4,/images/41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1655,572,1763,695


In [62]:
test_df["filepath"].nunique()

120

In [63]:
test_df["class"].value_counts()

red blood cell    5614
ring               169
trophozoite        111
gametocyte          12
schizont            11
difficult            5
Name: class, dtype: int64

In [64]:
test_df.filepath = test_df.filepath.apply(create_filename)
test_df.rename(columns={"filepath": "filename"}, inplace=True)
test_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1744,576,1883,708
1,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1249,863,1373,977
2,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1573,210,1711,335
3,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1124,1017,1253,1142
4,41be1bd3-0d31-4881-bf1f-3ccdfa21ff12.jpg,1944,1383,red blood cell,1655,572,1763,695


In [65]:
test_df.to_csv("images/annotation_test.csv",index = False)

In [ ]:
# generate labelmap
## https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/blob/master/training/labelmap.pbtxt

# generate tfrecord for API
## https://github.com/abdelrahman-gaber/tf2-object-detection-api-tutorial/blob/master/data_gen/generate_tfrecord.py

# pretrained models
## https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md